In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
import h5py
from numpy import dtype

# This notebook will be used to:
- #### Aggregate PSRC's TAZ to the ZIPCODEs
- #### Get the OD matrices to the ZIP-code level

# Aggregating PSRC's TAZs

First I'm gonna assign any TAZ that has its centroid inside the Zip code to that zipcode
After that, as there are some that have funny borders with the water, I'm gonna do the ones that intersect the Zipcode be assigned to that zipcode

In [82]:
#Importing the TAZ
TAZ = gpd.read_file('data/PSRC/soundcast_042125.gdb', layer = 'TAZ_2010')
TAZ = TAZ.drop(columns = ['park', 'acres', 'GlobalID', 'Shape_Length', 'Shape_Area'])
TAZ_original = TAZ.copy()
print(TAZ_original.shape[0])

#Importing the zipcodes in the 2010 format
ZIP_code = gpd.read_file('data/ZIP Codes/zip10_full.gpkg')
ZIP_code = ZIP_code[['GEOID10', 'geometry']]
ZIP_code_original = ZIP_code.copy()

3700


In [3]:
ZIP_code

,GEOID10,geometry
0,35442,"MULTIPOLYGON (((-88.25262 32.92675, -88.24972 ..."
1,85365,"MULTIPOLYGON (((-114.68466 32.68739, -114.6760..."
2,71973,"MULTIPOLYGON (((-94.46643 34.33074, -94.46638 ..."
3,95445,"MULTIPOLYGON (((-123.64305 38.85123, -123.6414..."
4,06870,"MULTIPOLYGON (((-73.58766 41.00765, -73.57197 ..."
...,...,...
33139,50565,"MULTIPOLYGON (((-95.09264 42.82368, -95.0923 4..."
33140,20566,"MULTIPOLYGON (((-77.0565 38.89767, -77.05362 3..."
33141,08562,"MULTIPOLYGON (((-74.66573 40.03401, -74.65859 ..."
33142,08069,"MULTIPOLYGON (((-75.50341 39.69856, -75.49624 ..."


In [4]:
epsg_work = 26910

In [5]:
#Making the centroids
TAZ['geometry'] = TAZ_original['geometry'].to_crs(epsg_work).centroid
TAZ


,taz,geometry
0,1.0,POINT (545537.557 5286129.922)
1,2.0,POINT (548122.031 5286344.538)
2,3.0,POINT (548718.019 5286633.57)
3,4.0,POINT (548718.484 5286031.037)
4,5.0,POINT (549379.446 5286544.365)
...,...,...
3695,3696.0,POINT (532401.006 5275608.358)
3696,3697.0,POINT (532469.443 5272860.464)
3697,3698.0,POINT (534285.03 5272495.224)
3698,3699.0,POINT (537281.343 5273180.28)


In [6]:
#Making the spatial join
ZIP_code = ZIP_code.to_crs(epsg = epsg_work)
TAZ_to_ZIP = gpd.sjoin(TAZ, ZIP_code, how='left', predicate='intersects')

#Counting if there is any NA
print(f'We have {TAZ_to_ZIP['GEOID10'].isna().sum()} TAZs that dont have a ZIP code')

Missing_TAZ = TAZ_to_ZIP[TAZ_to_ZIP['GEOID10'].isna()]

We have 94 TAZs that dont have a ZIP code


In [8]:
m = folium.Map(location= [47.605744, -122.334862])
#Adding TAZs
folium.GeoJson(
    TAZ_to_ZIP[TAZ_to_ZIP['GEOID10'].isna()],
    popup= folium.GeoJsonPopup(
        fields = ['taz']
    )
).add_to(m)

folium.GeoJson(
    ZIP_code[ZIP_code['GEOID10'].isin(TAZ_to_ZIP['GEOID10'])],
    popup= folium.GeoJsonPopup(
        fields = ['GEOID10']
    )
).add_to(m)

In [9]:
# m

In [11]:
Missing_TAZ_a = TAZ_original.loc[
    TAZ_original['taz'].isin(Missing_TAZ['taz'])].to_crs(epsg_work)

TAZ_to_ZIP_missing = gpd.sjoin(Missing_TAZ_a, ZIP_code, how='left', predicate='intersects')

TAZ_to_ZIP_missing = TAZ_to_ZIP_missing.groupby('taz').first().reset_index()

TAZ_to_ZIP_NONA = TAZ_to_ZIP.dropna()
TAZ_to_ZIP_NONA.loc[:,'geometry'] = TAZ_original[TAZ_original['taz'].isin(TAZ_to_ZIP_NONA['taz'])].loc[:,'geometry']
TAZ_to_ZIP_NONA

,taz,geometry,index_right,GEOID10
1,2.0,"MULTIPOLYGON (((1265788.093 268180.249, 126577...",20801.0,98133
2,3.0,"MULTIPOLYGON (((1265848.689 271492.146, 126617...",20801.0,98133
3,4.0,"MULTIPOLYGON (((1267095.215 269151.901, 126728...",20801.0,98133
4,5.0,"MULTIPOLYGON (((1270732.726 269633.63, 1271063...",20801.0,98133
5,6.0,"MULTIPOLYGON (((1271071.927 270062.991, 127106...",20801.0,98133
...,...,...,...,...
3694,3695.0,"MULTIPOLYGON (((1220136.14 239284.827, 1220912...",20787.0,98110
3695,3696.0,"MULTIPOLYGON (((1213639.678 239372.617, 121371...",20787.0,98110
3696,3697.0,"MULTIPOLYGON (((1212031.956 234139.577, 121203...",20787.0,98110
3697,3698.0,"MULTIPOLYGON (((1219221.496 231349.083, 121923...",20787.0,98110


In [22]:
TAZ_to_ZIP = pd.concat([TAZ_to_ZIP_NONA, TAZ_to_ZIP_missing])
# TAZ_to_ZIP.isna().sum()
TAZ_to_ZIP.shape[0]
#WE GOT EM BOSS

C:\Users\olive\PycharmProjects\OD_matrix\.venv\Lib\site-packages\geopandas\array.py:1754: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as NAD83 / UTM zone 10N (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


3700

In [30]:
#List of all the ZIP codes
Unique_ZIPS = TAZ_to_ZIP['GEOID10'].unique()
Unique_ZIPS = sorted(Unique_ZIPS.astype(int).tolist())
Unique_ZIPS

[98001,
 98002,
 98003,
 98004,
 98005,
 98006,
 98007,
 98008,
 98010,
 98011,
 98012,
 98014,
 98019,
 98020,
 98021,
 98022,
 98023,
 98024,
 98026,
 98027,
 98028,
 98029,
 98030,
 98031,
 98032,
 98033,
 98034,
 98036,
 98037,
 98038,
 98039,
 98040,
 98042,
 98043,
 98045,
 98047,
 98050,
 98051,
 98052,
 98053,
 98055,
 98056,
 98057,
 98058,
 98059,
 98065,
 98068,
 98070,
 98072,
 98074,
 98075,
 98077,
 98087,
 98092,
 98101,
 98102,
 98103,
 98104,
 98105,
 98106,
 98107,
 98108,
 98109,
 98110,
 98112,
 98115,
 98116,
 98117,
 98118,
 98119,
 98121,
 98122,
 98125,
 98126,
 98133,
 98134,
 98136,
 98144,
 98146,
 98148,
 98155,
 98158,
 98166,
 98168,
 98177,
 98178,
 98188,
 98195,
 98198,
 98199,
 98201,
 98203,
 98204,
 98205,
 98207,
 98208,
 98223,
 98241,
 98251,
 98252,
 98258,
 98270,
 98271,
 98272,
 98275,
 98282,
 98288,
 98290,
 98292,
 98294,
 98296,
 98303,
 98304,
 98310,
 98311,
 98312,
 98314,
 98315,
 98321,
 98323,
 98327,
 98328,
 98329,
 98332,
 98333,


In [39]:
#Creating a dictionary where the keys are the Zipcodes and the values are the vectors that have the TAZs assigned to that zipcode
ZIP_dict = {}
for ZIP in Unique_ZIPS:
    ZIP_dict[f'{ZIP}'] = TAZ_to_ZIP.loc[TAZ_to_ZIP['GEOID10'] == str(ZIP), 'taz'].tolist()


[1160.0,
 1161.0,
 1168.0,
 1169.0,
 1172.0,
 1173.0,
 1174.0,
 1178.0,
 1180.0,
 1181.0,
 1182.0,
 1183.0,
 1184.0,
 1186.0,
 1192.0,
 1194.0,
 1196.0,
 1197.0,
 1198.0,
 1199.0,
 1216.0]

# Assigning the OD to each zipcode

In [27]:
#Creating the OD matrix
OD_matrix = pd.DataFrame(
    np.zeros((len(Unique_ZIPS), len(Unique_ZIPS)), dtype=float),
    index = Unique_ZIPS,
    columns = Unique_ZIPS
)
OD_matrix

,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,...,98443,98444,98445,98446,98465,98466,98467,98498,98499,98580
98001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
#Loading the OD thing
OD_PSRC = h5py.File('data/PSRC/10to14.h5', 'r')
OD_SOV = pd.DataFrame(OD_PSRC['sov_inc2'][:,:])
OD_SOV

,0,1,2,3,4,5,6,7,8,9,...,3855,3856,3857,3858,3859,3860,3861,3862,3863,3864
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,7.817399e-07,4.574547e-10,6.089718e-08,0.000001,2.390244e-08,1.306370e-07,1.950710e-07,4.099306e-08,3.806920e-07,1.892043e-16
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.030055e-06,6.046429e-10,8.040695e-08,0.000002,3.155793e-08,1.851400e-07,2.577165e-07,5.412299e-08,5.015683e-07,2.605149e-16
2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.179136e-06,6.867987e-10,9.146252e-08,0.000002,3.587680e-08,2.449523e-07,2.926014e-07,6.158687e-08,5.690270e-07,2.832674e-16
3,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.596329e-06,9.375857e-10,1.246042e-07,0.000003,4.860602e-08,2.505180e-07,3.996688e-07,8.388512e-08,7.778181e-07,2.924039e-16
4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.476411e-06,8.600105e-10,1.144194e-07,0.000002,4.481072e-08,2.751946e-07,3.661550e-07,7.707008e-08,7.105344e-07,3.216626e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3860,1.306370e-07,1.851400e-07,2.449523e-07,2.505180e-07,2.751946e-07,3.448834e-07,2.833713e-07,3.401829e-07,3.919607e-07,4.152480e-07,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3861,1.950710e-07,2.577165e-07,2.926014e-07,3.996688e-07,3.661550e-07,3.774394e-07,4.742299e-07,5.375614e-07,4.911217e-07,5.403968e-07,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3862,4.099306e-08,5.412299e-08,6.158687e-08,8.388512e-08,7.707008e-08,7.939290e-08,9.998936e-08,1.135222e-07,1.033013e-07,1.136654e-07,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3863,3.806920e-07,5.015683e-07,5.690270e-07,7.778181e-07,7.105344e-07,7.173485e-07,9.176323e-07,1.024050e-06,9.284592e-07,1.021927e-06,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [64]:
OD_SOV.loc[ZIP_dict['98001'], ZIP_dict['98002']].sum().sum()
for zip1, rows in ZIP_dict.items():
    for zip2, cols in ZIP_dict.items():
        OD_matrix.loc[zip1, zip2] = OD_SOV.loc[rows, cols].values.sum()

In [83]:
OD_SOV.loc[~OD_SOV.index.isin(TAZ_to_ZIP['taz'].tolist())].index.tolist()

[0,
 3701,
 3702,
 3703,
 3704,
 3705,
 3706,
 3707,
 3708,
 3709,
 3710,
 3711,
 3712,
 3713,
 3714,
 3715,
 3716,
 3717,
 3718,
 3719,
 3720,
 3721,
 3722,
 3723,
 3724,
 3725,
 3726,
 3727,
 3728,
 3729,
 3730,
 3731,
 3732,
 3733,
 3734,
 3735,
 3736,
 3737,
 3738,
 3739,
 3740,
 3741,
 3742,
 3743,
 3744,
 3745,
 3746,
 3747,
 3748,
 3749,
 3750,
 3751,
 3752,
 3753,
 3754,
 3755,
 3756,
 3757,
 3758,
 3759,
 3760,
 3761,
 3762,
 3763,
 3764,
 3765,
 3766,
 3767,
 3768,
 3769,
 3770,
 3771,
 3772,
 3773,
 3774,
 3775,
 3776,
 3777,
 3778,
 3779,
 3780,
 3781,
 3782,
 3783,
 3784,
 3785,
 3786,
 3787,
 3788,
 3789,
 3790,
 3791,
 3792,
 3793,
 3794,
 3795,
 3796,
 3797,
 3798,
 3799,
 3800,
 3801,
 3802,
 3803,
 3804,
 3805,
 3806,
 3807,
 3808,
 3809,
 3810,
 3811,
 3812,
 3813,
 3814,
 3815,
 3816,
 3817,
 3818,
 3819,
 3820,
 3821,
 3822,
 3823,
 3824,
 3825,
 3826,
 3827,
 3828,
 3829,
 3830,
 3831,
 3832,
 3833,
 3834,
 3835,
 3836,
 3837,
 3838,
 3839,
 3840,
 3841,
 3842,
 3